In [1]:
import torch
from torchvision import transforms
from PIL import Image
from transformers import DistilBertTokenizer
from IPython.display import Image as IPImage, display

# Force reload model module (in case of cached old version)
import importlib
import model as model_module
importlib.reload(model_module)
from model import ViralPredictor

# --- CONFIGURATION ---
MODEL_PATH = "data/models/viral_predictor.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Define transformations (MUST match training transforms)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# 2. Model loading function
def load_model():
    print(f"Loading model from {MODEL_PATH}...")
    model = ViralPredictor()
    # Load weights (state_dict)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()  # IMPORTANT: Switches model to test mode (disables Dropout etc.)
    return model


# 3. Prediction function (Inference)
def predict_viral(model, image_path, title_text):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    # A. Image Preparation
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        return "Error: Image file not found."

    # Add batch dimension (Model expects [1, 3, 224, 224], not [3, 224, 224])
    image_tensor = transform(image).unsqueeze(0).to(DEVICE)

    # B. Text Preparation
    text_inputs = tokenizer(
        title_text,
        return_tensors="pt",
        padding='max_length',
        truncation=True,
        max_length=50
    )
    input_ids = text_inputs['input_ids'].to(DEVICE)
    attention_mask = text_inputs['attention_mask'].to(DEVICE)

    # C. Prediction
    with torch.no_grad():  # Disable gradient computation (saves memory)
        prediction = model(image_tensor, input_ids, attention_mask)

    # Extract number from tensor
    score = prediction.item()
    return score


def print_verdict(score, title, image_path):
    """Helper function to print prediction results."""
    print(f"\nTitle: {title}")
    print(f"File: {image_path}")
    display(IPImage(filename=test_image, width=400))
    print(f"Viral Probability: {score:.2%}")

    if score > 0.6:
        print(">> VERDICT: 🔥 POTENTIAL VIRAL!")
    elif score < 0.4:
        print(">> VERDICT: 💀 WEAK...")
    else:
        print(">> VERDICT: 😐 Average")



In [2]:
model = load_model()

print("-" * 30)
print("VIRALITY TESTER")
print("-" * 30)

------------------------------
VIRALITY TESTER
------------------------------


In [3]:
# Test 1
test_image = "data/test/test1.jpg"
test_title = "МОРГЕНШТЕРН — Группа Крови (Live in Poland 2025) Полная версия"
score = predict_viral(model, test_image, test_title)
print_verdict(score, test_title, test_image)

Viral Probability: 57.21%
>> VERDICT: 😐 Average


In [4]:
# Test 2
test_image = "data/test/test2.jpg"
test_title = "How I Transformed VS Code Into a Cozy Workspace #devsetup"
score = predict_viral(model, test_image, test_title)
print_verdict(score, test_title, test_image)


Viral Probability: 72.66%
>> VERDICT: 🔥 POTENTIAL VIRAL!


In [5]:
# Test 3
test_image = "data/test/test3.jpg"
test_title = "How I Code Profitable Apps SOLO (Idea + Build + Marketing Guide)"
score = predict_viral(model, test_image, test_title)
print_verdict(score, test_title, test_image)


Viral Probability: 80.88%
>> VERDICT: 🔥 POTENTIAL VIRAL!


In [6]:
# Test 4
test_image = "data/test/test4.jpeg"
test_title = 'Michael Jackson\'s Drummer Jonathan Moffett Performs "Smooth Criminal"'
score = predict_viral(model, test_image, test_title)
print_verdict(score, test_title, test_image)


Viral Probability: 77.32%
>> VERDICT: 🔥 POTENTIAL VIRAL!


In [7]:
# Test 5
test_image = "data/test/test5.jpg"
test_title = 'Year summary 2025'
score = predict_viral(model, test_image, test_title)
print_verdict(score, test_title, test_image)


Viral Probability: 21.56%
>> VERDICT: 💀 WEAK...
